In [2]:
#John David Conley
#Deep Learning ICP 10
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

from google.colab import files
uploaded = files.upload()


Mounted at /content/drive


Saving Sentiment.csv to Sentiment.csv


1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are 
happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)
2. Apply GridSearchCV on the source code provided in the class

In [3]:
#Provided
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)
model.save('sentiModel')


291/291 - 67s - loss: 0.8380 - accuracy: 0.6357 - 67s/epoch - 230ms/step
144/144 - 2s - loss: 0.7984 - accuracy: 0.6621 - 2s/epoch - 11ms/step
0.7983637452125549
0.6620795130729675
['loss', 'accuracy']


In [4]:
#1
from keras.models import load_model
import numpy as np
reModel = load_model('sentiModel')
txt="A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"
seq = tokenizer.texts_to_sequences([txt])
padded = pad_sequences(seq, maxlen=28)
pred = model.predict(padded)
print(pred)

1/1 [==============================] - 1s 519ms/step
[[0.4972104 0.1098179 0.3929716]]


In [ ]:
#2 (no GPU)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from keras.wrappers.scikit_learn import KerasClassifier
model2=KerasClassifier(build_fn=lambda:model,verbose=0)
batch_size=[10,20,40]
epochs=[1,2,3]
param_grid=dict(batch_size=batch_size,epochs=epochs)
#grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(accuracy_score))
#grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(accuracy_score(y,np.argmax(X_train, axis=1))))
grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(np.argmax(X_train, axis=1)))
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

<ipython-input-83-28c1acf94e40>:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model2=KerasClassifier(build_fn=lambda:model,verbose=0)


59/59 [==============================] - 2s 36ms/step


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
TypeError: 'numpy.ndarray' object is not callable

  warnings.warn(


59/59 [==============================] - 1s 22ms/step


In [ ]:
#2 (GPU)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from keras.wrappers.scikit_learn import KerasClassifier
model2=KerasClassifier(build_fn=lambda:model,verbose=0)
batch_size=[10,20,40]
epochs=[1,2,3]
param_grid=dict(batch_size=batch_size,epochs=epochs)
#grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(accuracy_score))
#grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(accuracy_score(y,np.argmax(X_train, axis=1))))
grid = GridSearchCV(estimator=model2,param_grid=param_grid,scoring=make_scorer(np.argmax(X_train, axis=1)))
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

59/59 [==============================] - 1s 9ms/step


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
TypeError: 'numpy.ndarray' object is not callable

  warnings.warn(


59/59 [==============================] - 1s 9ms/step
